# Align robot model

In [ ]:
# show pointclouds
import open3d as o3d
import cv2
import numpy as np
import json
from os.path import join, abspath, dirname
import matplotlib.pyplot as plt
import yaml
%matplotlib inline

In [ ]:
def pcdCamera(scale=1.0, color=[1, 0, 0]):
    points = [[-1, 0.75, 0],[1, 0.75, 0],[1, -0.75, 0],[-1, -0.75, 0],[0,0,-0.6]]
    lines = [[0,1],[1,2],[2,3],[3,0],
             [0,4],[1,4],[2,4],[3,4]]
    colors = [color for i in range(len(lines))]
    line_set = o3d.geometry.LineSet()

    points = np.array(points)*scale
    line_set.points = o3d.utility.Vector3dVector(points)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector(colors)
    return line_set

def axis(scale=1.0):
    points = [[0,0,0],[1,0,0],[0,1,0],[0,0,1]]
    lines = [[0, i] for i in range(1,4)]
    colors = [[i==0,i==1,i==2] for i in range(len(lines))]
    line_set = o3d.geometry.LineSet()

    points = np.array(points)*scale
    line_set.points = o3d.utility.Vector3dVector(points)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector(colors)
    return line_set

# Load camera poses and robot model

In [ ]:
CAM_NUM = 4

FOLDER = '../data'
robot_file = join(FOLDER, 'robot.ply')
print("robot file:", robot_file)

fs_read = cv2.FileStorage(join(FOLDER, 'final_camera_poses.yml'), cv2.FILE_STORAGE_READ)
T = {}
for key in fs_read.root().keys():
    print("camera pose:", key)
    T[key] = fs_read.getNode(key).mat()

## Visualize robot model and camera poses

In [ ]:
robot_pcd = o3d.io.read_point_cloud(robot_file)
o3d.visualization.draw_geometries([robot_pcd.voxel_down_sample(voxel_size=0.005), axis(0.5)])

## Show images for reference

In [ ]:
cam_pcds = []
for i in range(CAM_NUM):
    cam_pcd = pcdCamera(0.1, color=[i==0,i==1,i==2])
    key = f"originimg{i}"
    cam_pcd.transform(T[key])
    cam_pcds.append(cam_pcd)
show_pcds = [*cam_pcds, axis(0.5)]
o3d.visualization.draw_geometries(show_pcds, width=1200, height=800)

In [ ]:
imgs = []
for i in range(CAM_NUM):
    fname = join(FOLDER, f'img{i}.jpg')
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    imgs.append(img)
    
fig, ax = plt.subplots(1, CAM_NUM, figsize=(12,4))
for i, (img, a) in enumerate(zip(imgs, ax)):
    a.imshow(img)
    a.set_yticklabels([])
    a.set_title(f"img{i}")


## Click points on robot

In [ ]:
def pick_points(pcd):
    print("")
    print("1) Please pick at point using [shift + left click]")
    print("   Press [shift + right click] to undo point picking")
    print("2) Afther picking points, press q for close the window")
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window(width=1200, height=800)
    vis.add_geometry(pcd)
    vis.run()  # user picks points
    vis.destroy_window()
    print("")
    return vis.get_picked_points()

In [ ]:
print(f'click at {CAM_NUM} cameras')
picked_id = pick_points(robot_pcd)
print(picked_id)

## Align by Umeyama's method
S. Umeyama, "Least-squares estimation of transformation parameters between two point patterns," in IEEE Transactions on Pattern Analysis and Machine Intelligence, vol. 13, no. 4, pp. 376-380, April 1991.
doi: 10.1109/34.88573  
https://ieeexplore.ieee.org/document/88573

Reference for implementation  
https://github.com/MichaelGrupp/evo/blob/5fce036136001c0ee7cb2d03e992bef9c5abe746/evo/core/geometry.py#L30

In [ ]:
# min ||pt_to - T*pt_from||^2
def align_pts(pt_from, pt_to, with_scale=True):
    x = pt_from.copy().T
    y = pt_to.copy().T
    assert x.shape == y.shape

    # m dim, n points
    m, n = x.shape
    mean_x = x.mean(axis=1)[:,np.newaxis]
    mean_y = y.mean(axis=1)[:,np.newaxis]

    # zero center
    x -= mean_x
    y -= mean_x
    sigma_x = np.mean(np.linalg.norm(x, axis=0)**2)
    
    # 
    cov_xy = np.zeros((m, m))
    for i in range(n):
        cov_xy += np.outer(y[:,i], x[:,i])
    cov_xy = cov_xy*1.0/n
    
    # svd
    u, d, v = np.linalg.svd(cov_xy)

    s = np.eye(m)
    if np.linalg.det(u) * np.linalg.det(v) < 0.0:
        s[m - 1, m - 1] = -1

    R = u.dot(s).dot(v)
    c = 1 / sigma_x * np.trace(np.diag(d).dot(s)) if with_scale else 1.0
    c = float(c)
    t = mean_y - c*R.dot(mean_x)
    t = t.flatten()
    return c, R, t

In [ ]:
# picked_id = [23660, 0, 30012, 40612]
robot_pcd = o3d.io.read_point_cloud(robot_file)
picked_pts = np.array(robot_pcd.points)[picked_id] # copy points

cam_pts = []
for i in range(CAM_NUM):
    key = f"originimg{i}"
    cam_pts.append(T[key][:3,3])
cam_pts = np.array(cam_pts)

# estimate matrix for alignment
c, R, t = align_pts(picked_pts, cam_pts)

# save pose
yml_file = join(FOLDER, "robot_align_matrix.yml")
fs = cv2.FileStorage(join(FOLDER, yml_file), cv2.FILE_STORAGE_WRITE)
fs.write("robot_scale", c)
fs.write("robot_rvec", cv2.Rodrigues(R)[0])
fs.write("robot_tvec", t)
fs.release()

## Visualize aligned pcd

In [ ]:
cam_pcds = []
for i in range(CAM_NUM):
    cam_pcd = pcdCamera(0.1, color=[i==0,i==1,i==2])
    key = f"originimg{i}"
    cam_pcd.transform(T[key])
    cam_pcds.append(cam_pcd)

    
robot_pcd = o3d.io.read_point_cloud(robot_file)
T_wr = np.eye(4)
T_wr[:3,:3] = c*R
T_wr[:3,3] = t
robot_pcd.transform(T_wr)
    
show_pcds = [*cam_pcds, robot_pcd, axis(0.5)]

for it in show_pcds:
    it.transform([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
o3d.visualization.draw_geometries(show_pcds, width=1200, height=800)

# Align LRF

In [ ]:
bev = cv2.imread(join(FOLDER, 'bev.jpg'))
data_xy = np.loadtxt(join(FOLDER, 'urg_xy.csv'), delimiter=",")

fs = cv2.FileStorage(join(FOLDER, 'bev_info.yml'), cv2.FileStorage_READ)
pixel_to_m = fs.getNode('pixel_to_m').real()
# np.savetxt(join(FOLDER, 'urg_xy_.csv'), data_xy, delimiter=",", fmt='%.5f')

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12,4))
ax[0].plot(data_xy[:,0], data_xy[:,1], 'cd')
ax[0].set_aspect(aspect=1)

ax[1].imshow(bev[:,:,::-1])

## Click points in LRF

In [ ]:
def LRFpcd(data_xy):
    num_pts = data_xy.shape[0]
    data_z = np.zeros((num_pts))
    xyz = np.stack([data_xy[:,0],data_xy[:,1], data_z], axis=1)
    lrf_pcd = o3d.geometry.PointCloud()
    lrf_pcd.points = o3d.utility.Vector3dVector(xyz)
    return lrf_pcd

In [ ]:
# generate 3D point cloud on floor
lrf_pcd = LRFpcd(data_xy)
o3d.visualization.draw_geometries([lrf_pcd, axis(0.5)], width=1200, height=800)

In [ ]:
picked_id = pick_points(lrf_pcd)
print(picked_id)

In [ ]:
# picked_id = [866, 1021, 204, 416]
picked_pts = np.array(lrf_pcd.points)[picked_id] # copy points

## Click points in bird's-eye view image

In [ ]:
r = 5
pts = []
def on_click(event, x, y, flags, param):
    global pts
    if event == cv2.EVENT_LBUTTONDOWN:
        pts.append((x, y))
        print(f'Added point ({x},{y})')
        
winname = 'click points'
cv2.namedWindow(winname)
cv2.setMouseCallback(winname, on_click)
while True:
    out = bev.copy()
    for pt in pts:
        cv2.circle(out, pt, r, (0,0,255), 5)

    cv2.imshow(winname, out)
    key = cv2.waitKey(100)
    if key==27:
        break
cv2.destroyAllWindows()
print('Total number of points:', len(pts))

In [ ]:
# world_pt.at<double>(0) = (u - uc)*pixel_to_m;
# world_pt.at<double>(1) = (-v + vc )*pixel_to_m;

In [ ]:
# pts = [(135, 180), (134, 271), (348, 374), (347, 259)]

# Convert to world coordinate
h, w, _ = bev.shape
bev_pts = []
for pt in pts:
    x = (pt[0] - w/2) * pixel_to_m
    y = (-pt[1] + h/2) * pixel_to_m    
    bev_pts.append([x, y])
bev_pts = np.array(bev_pts)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(bev_pts[:,0], bev_pts[:,1], 'cd')
plt.plot(picked_pts[:,0], picked_pts[:,1], 'rd')
ax.set_aspect(aspect=1)

## Align them together

In [ ]:
picked_pts = picked_pts[:,:2]
# estimate matrix for alignment
c, R, t = align_pts(picked_pts, bev_pts, False)

print(c)
print(R)
print(t)

# save
yml_file = join(FOLDER, "lrf_align_matrix.yml")
fs = cv2.FileStorage(join(FOLDER, yml_file), cv2.FILE_STORAGE_WRITE)
fs.write("lrf_rot", np.arctan2(R[1,0], R[0, 0]))
fs.write("lrf_tvec", t)
fs.release()

In [ ]:
lrf_pcd = LRFpcd(data_xy)
T = np.eye(4)
T[:2,:2] = c*R
T[:2,3] = t
lrf_pcd.transform(T)
    
show_pcds = [lrf_pcd, axis(0.5)]
show_pcds = [lrf_pcd, axis(0.5), robot_pcd]


for it in show_pcds:
    it.transform([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
o3d.visualization.draw_geometries(show_pcds, width=1200, height=800)